In [137]:
# On importe les librairies nécessaires
import dotenv # Pour lire nos variables environnements avec nos APIs

# On importe quelques librairies de manipulation de données
import numpy as np
import pandas as pd
import re
import os

# On importe les modules nécessaires de LangChain
from langchain.chains import RetrievalQA, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub, HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
import warnings
warnings.filterwarnings('ignore')

import datetime
import json
tod_date = datetime.date.today()
tod_hour = datetime.datetime.now()

In [138]:
# On lit nos variables environnments avec nos clés APIs
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [139]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_new_tokens":500})

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [141]:
def remove_after_last_brace(text):
    # Index de la dernière accolade fermante
    last_brace_index = -1

    for i, char in enumerate(text):
        if char == '}':
            last_brace_index = i
    json_str = text[:last_brace_index+1]
    return json.loads(json_str)

In [222]:
temp1 = """[INST]
        Tu dois extraire des informations de la phrase données.

        N'invente pas, et extrais dans un JSON valide la VILLE et la DATE et l'HEURE. Si tu ne sait pas, met 'None'.
        l'HEURE doit etre une heure valide.
        Aujourd'hui, nous sommes le {0} à {1}h.

        Le JSON doit avoir ce format:
        (
        "ville":"ville",
        "date":"YYYY/MM/DD",
        "heure":"HH"
        )

        ----- 
""".format(tod_date.strftime('%Y/%m/%d'),tod_hour.strftime('%H'))
temp2 = """
        Voici la requête :
            {query}

            [/INST]
        JSON:
"""

templ = temp1 + temp2

query = "Quelle meteo pour demain soir à Lyon?"

# On instancie notre template de prompt où l'on indique que nos deux variables entrantes sont le contexte (documents) et la requête (question)
promp_rag = PromptTemplate(input_variables=["query"], template=templ)
chain = LLMChain(prompt=promp_rag, llm=llm,verbose=False)
response = chain.invoke({"query": query})
answer = response["text"].split("JSON:")[1]
json_fin = remove_after_last_brace(answer)

# On le place dans une variable pour indiquer que ce sera le prompt de notre retriever
print(json_fin)

{'ville': 'Lyon', 'date': '2024/03/14', 'heure': '18'}


In [225]:
def clean_json(json_object,date,hour):
    tod_h = hour.strftime('%H')
    new_json = json_object
    if list(json_fin.values()) == ['None','None','None']:
        return "Need to re-ask"
    if json_object["ville"] == 'None':
        new_json["ville"] = "Lyon"
    
    day_plus_five = date + datetime.timedelta(days=5)
    if json_object["heure"] < tod_h : 
        json_object["heure"] = tod_h + 1
    if (json_object["date"] > day_plus_five.strftime('%Y/%m/%d')) & (json_object["date"] != 'None'):        
        new_json["date"] = day_plus_five.strftime('%Y/%m/%d')
    
    return new_json

In [228]:
new_json = clean_json(json_fin,tod_date,tod_hour)
new_json

{'ville': 'Lyon', 'date': '2024/03/14', 'heure': '18'}